In [1]:
from neuron import h, units, rxd, load_mechanisms
import neuron
import matplotlib
from matplotlib import pyplot as plt
h.load_file("stdrun.hoc")
%matplotlib inline
from itertools import product
import pandas as pd
import os
import numpy as np
import math

--No graphics will be displayed.


In [2]:
class Islet:
    def __init__(self, id, prob_alpha, prob_beta, islet_radius, num_cells):
        """Initialize islet instance"""
        # path to initialization file, mechanisms, output, and generation identifier
        self._id = id
        self.islet_radius = islet_radius
        self.num_cells = num_cells
        self.prob_alpha = prob_alpha
        self.prob_beta = prob_beta
        self.num_betas = math.floor(self.num_cells * self.prob_beta)
        self.num_alphas = math.ceil(self.num_cells * self.prob_alpha)
        self.num_deltas = self.num_cells - self.num_betas - self.num_alphas

    def __repr__(self):
        return 'Islet[{}]'.format(self._id)

    def spatial_setup(self):
        nx, ny, nz = (10,10,10)
        # The below statements take a line from -radius to radius and chops it up into n segments
        x_coords = np.linspace(-self.islet_radius, self.islet_radius, nx)
        y_coords = np.linspace(-self.islet_radius, self.islet_radius, ny)
        z_coords = np.linspace(-self.islet_radius, self.islet_radius, nz)
        self.locations = []
        count = 0
        while count < self.num_cells:
            # Select a random value for x, y, and z coordinates
            x,y,z = np.random.choice(x_coords, 1), np.random.choice(y_coords, 1), np.random.choice(z_coords, 1)
            # Check if the point is within the islet sphere
            if math.sqrt(x**2 + y**2 + z**2) < self.islet_radius:
                # Check if random coordinates chosen are unique so not two cells have the same location
                if any(location == [x[0],y[0],z[0]] for location in self.locations): 
                    continue
                else:
                    # If location unique, append it to the locations list
                    self.locations.append((x[0],y[0],z[0]))
                    count+=1

    def populate_cells(self):
        # Create cell dictionary containing all the sections in the islet
        self.cells = {}
       # Create the sections, which can be accessed by their name in the cells dictionary
        for i in range(self.num_alphas):
            self.cells["A_" + str(i)] = h.Section(name = "A_" + str(i))
        for i in range(self.num_betas):
            self.cells["B_" + str(i)] = h.Section(name = "B_" + str(i))
        for i in range(self.num_deltas):
            self.cells["D_" + str(i)] = h.Section(name = "D_" + str(i))
        # Insert one mechanism into each cell. In future make this like original setup
        # with config file and mechanism names for each cell type.
        for cell in self.cells:
            self.cells[cell].insert('one')

    def set_cell_locations(self):
        # Separate the cells dictionary into lists for each cell type
        # This is accomplished by using the search criteria of if "A", "B", or "D"
        # is in cell name
        alpha_sections = [val for key, val in self.cells.items() if "A" in key]
        beta_sections = [val for key, val in self.cells.items() if "B" in key]
        delta_sections = [val for key, val in self.cells.items() if "D" in key]
        count = 0
        for i in range(self.num_cells):
            while count < self.num_alphas:
                print("Setting alpha cell locations")
                alpha_sections[i].pt3dclear()
                alpha_sections[i].pt3dadd(self.locations[i][0], self.locations[i][1], self.locations[i][2], 7)
                alpha_sections[i].pt3dadd(self.locations[i][0] + 7, self.locations[i][1], self.locations[i][2], 7)
                count+=1
            while self.num_alphas <= count < self.num_alphas + self.num_betas:
                print("Setting beta cell locations")
                beta_sections[i].pt3dclear()
                beta_sections[i].pt3dadd(self.locations[i][0], self.locations[i][1], self.locations[i][2], 15.5)
                beta_sections[i].pt3dadd(self.locations[i][0] + 15.5, self.locations[i][1], self.locations[i][2], 15.5)
                count+=1
            while self.num_alphas + self.num_betas <= count < self.num_cells:
                print("Setting delta cell locations")
                delta_sections[i].pt3dclear()
                delta_sections[i].pt3dadd(self.locations[i][0], self.locations[i][1], self.locations[i][2], 7)
                delta_sections[i].pt3dadd(self.locations[i][0] + 7, self.locations[i][1], self.locations[i][2], 7)   
                count+=1            

In [71]:
# This function first creates a matrix that contains the distance between each cell.
# Each row of this matrix corresponds to a cell.
# Say there are k alpha cells, l beta cells, and m delta cells.
# And let n=k+l+m be the total number of cells.
# The first k rows correspond to alpha cells.
# Rows k+1 to k+l correspond to beta cells.
# Rows k+l+1 to n correspond to delta cells.
# This matrix will then be subsetted into different matrices
# One for beta cells affecting alphas that will contain the rows corresponding to 
# beta cells and the columns corresponding to alpha cells (D_ba).
# One for delta cells affecting alphas that will contain the rows corresponding to 
# delta cells and the columns corresponding to alpha cells (D_da).
# One for alpha cells affecting betas that will contain the rows corresponding to 
# alpha cells and the columns corresponding to beta cells (D_ab).
# One for delta cells affecting betas that will contain the rows corresponding to 
# delta cells and the columns corresponding to beta cells (D_db).
# One for beta cells affecting deltas that will contain the rows corresponding to 
# beta cells and the columns corresponding to delta cells (D_bd).
def create_dist_matrices(islet_name):
    # Create a numpy array containing the distance between each cell
    dist_matrix = np.array([math.dist(location_1, location_2) for location_1 in islet_name.locations for location_2 in islet_name.locations])
    # Reshape the array to be nxn where each row corresponds to a single cell and the 
    # distances between itself and all other cells
    dist_matrix = np.reshape(dist_matrix, (islet_name.num_cells, islet_name.num_cells))
    dist_matrix = np.arange(100)
    dist_matrix.shape = (10,10)
    # Add the identity matrix so that the diagonal values are 1
    #dist_matrix = dist_matrix + np.identity(islet_name.num_cells)
    # Create D_ba
    D_ba = dist_matrix[0:islet_name.num_alphas, islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas]
    # Create D_da
    D_da = dist_matrix[0:islet_name.num_alphas, islet_name.num_alphas+islet_name.num_betas:islet_name.num_cells]
    # Create D_ab
    D_ab = dist_matrix[islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas, 0:islet_name.num_alphas]
    # Create D_db
    D_db = dist_matrix[islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas, islet_name.num_alphas+islet_name.num_betas:islet_name.num_cells]
    # Create D_bd
    D_bd = dist_matrix[islet_name.num_alphas+islet_name.num_betas:islet_name.num_cells, islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas]
    return dist_matrix,D_ba,D_da,D_ab,D_db,D_bd

In [22]:
test_np_array = np.arange(16)
test_np_array.shape = (4,4)
test_np_array

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [26]:
test_np_array[0:2,0:3]

array([[0, 1, 2],
       [4, 5, 6]])

In [72]:
# id, prob_alpha, prob_beta, islet_radius, num_cells)
test_islet = Islet(1,0.3,0.5,1,10)
test_islet.spatial_setup()
test_islet.populate_cells()
test_islet.set_cell_locations()

Setting alpha cell locations
Setting alpha cell locations
Setting alpha cell locations
Setting beta cell locations
Setting beta cell locations
Setting beta cell locations
Setting beta cell locations
Setting beta cell locations
Setting delta cell locations
Setting delta cell locations


In [73]:
test_islet.locations

[(-0.7777777777777778, -0.33333333333333337, -0.33333333333333337),
 (0.5555555555555554, 0.11111111111111116, -0.11111111111111116),
 (-0.33333333333333337, 0.5555555555555554, 0.11111111111111116),
 (0.5555555555555554, -0.11111111111111116, 0.7777777777777777),
 (-0.11111111111111116, -0.11111111111111116, 0.11111111111111116),
 (0.11111111111111116, -0.33333333333333337, -0.33333333333333337),
 (0.11111111111111116, 0.5555555555555554, -0.7777777777777778),
 (0.11111111111111116, 0.11111111111111116, -0.5555555555555556),
 (0.7777777777777777, 0.11111111111111116, 0.5555555555555554),
 (0.11111111111111116, -0.33333333333333337, -0.5555555555555556)]

In [74]:
dist_matrix,D_ba,D_da,D_ab,D_db,D_bd = create_dist_matrices(test_islet)

In [75]:
dist_matrix

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [76]:
D_ba

array([[ 3,  4,  5,  6,  7],
       [13, 14, 15, 16, 17],
       [23, 24, 25, 26, 27]])

In [77]:
D_da

array([[ 8,  9],
       [18, 19],
       [28, 29]])

In [78]:
D_ab

array([[30, 31, 32],
       [40, 41, 42],
       [50, 51, 52],
       [60, 61, 62],
       [70, 71, 72]])

In [79]:
D_db

array([[38, 39],
       [48, 49],
       [58, 59],
       [68, 69],
       [78, 79]])

In [80]:
D_bd

array([[83, 84, 85, 86, 87],
       [93, 94, 95, 96, 97]])

In [5]:
test_islet.cells["B_0"].L

15.5

In [9]:
test_islet.cells["B_0"](0.5).one.JIS

0.0

In [16]:
list(test_islet.cells.keys())

['A_0', 'B_0', 'D_0']

In [ ]:
# Function to make column vectors of JIS, JGS, and JSS values
# These vectors will need to be populated with new values at each time step
def create_secretion_rate_matrix(islet_name):
    # Create list of cell names by accessing keys in cell
    # dictionary
    cell_names = list(islet_name.cells.keys())
    # Initialize column vectors
    JIS_col_matrix = []
    JGS_col_matrix = []
    JSS_col_matrix = []
    # Add JIS, JGS, JSS values to corresponding vector based on name
    for cell in cell_names:
        if "B" in cell:
            JIS_col_matrix.append(islet_name.cells[cell](0.5).one.JIS)
        if "A" in cell:
            JGS_col_matrix.append(islet_name.cells[cell](0.5).one.JGS)
        if "D" in cell:
            JSS_col_matrix.append(islet_name.cells[cell](0.5).one.JSS)

In [17]:
test_list = ["A", "B", "D"]
test_list.append("Q")

In [18]:
test_list

['A', 'B', 'D', 'Q']

In [31]:
# Cell classes aren't really worth what we are doing.

# class Alpha(Islet):
#     def __init__(self, id, x, y, z):
#         self._id = id
#         self.cell = h.Section(name = "A" + str(id), cell = self)
#         self.cell.pt3dclear()
#         self.cell.pt3dadd(x,y,z,7)
#         self.cell.pt3dadd(x+7, y,z,7)
#     def __repr__(self):
#         return 'Alpha[{}]'.format(self._id)

# class Beta(Islet):
#     def __init__(self, id, x, y, z):
#         self._id = id
#         self.cell = h.Section()
#         self.cell.pt3dclear()
#         self.cell.pt3dadd(x,y,z,15.5)
#         self.cell.pt3dadd(x+15.5, y,z,15.5)
#     def __repr__(self):
#         return 'Beta[{}]'.format(self._id)

# class Delta(Islet):
#     def __init__(self, id, x, y, z):
#         self._id = id
#         self.cell = h.Section()
#         self.cell.pt3dclear()
#         self.cell.pt3dadd(x,y,z,7)
#         self.cell.pt3dadd(x+7, y,z,7)
#     def __repr__(self):
#         return 'Delta[{}]'.format(self._id)